Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [23]:
[1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5]

[0.01, 0.005, 0.001, 0.0005, 0.0001, 5e-05, 1e-05]

In [66]:
import math

batch_size = 128
hidden_units = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units], stddev=1.0/math.sqrt(float(image_size * image_size))))
  biases = tf.Variable(tf.zeros([hidden_units]))
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  layer1 = tf.nn.dropout(layer1, 0.5)
  layer1v = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  layer1t = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)

  ws = tf.Variable(tf.truncated_normal([hidden_units, num_labels], stddev=1.0/math.sqrt(float(hidden_units))))
  bs = tf.Variable(tf.zeros([num_labels]))
  layer2 = tf.matmul(layer1, ws) + bs
  layer2v = tf.matmul(layer1v, ws) + bs
  layer2t = tf.matmul(layer1t, ws) + bs
  
  # Training computation.
  logits = layer2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # regularization
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(ws))

  #[1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6]
  #r = 0    # 90.6%, V 88.1%, T 94.3%
  #r = 5e-3 # 86.7%, V 84.8%, T 91.6%
  #r = 1e-3 # 88.3%, V 87.2%, T 93.9%
  #r = 5e-4 # 89.1%, V 87.9%, T 94.2%
  #r = 1e-4 # 89.1%, V 88.1%, T 94.6%
  #r = 5e-5 # 87.5%, V 88.1%, T 94.2%
  #r = 1e-5 # 89.8%, V 88.2%, T 94.7%
  #r = 5e-6 # 89.1%, V 88.0%, T 94.3%

  # overfitting
  #r = 0    # 100.0%, V 80.2%, T 87.4%
  #r = 5e-3 # 100.0%, V 80.3%, T 87.8%
  #r = 1e-3 # 100.0%, V 80.3%, T 87.7%
  #r = 5e-4 # 100.0%, V 80.4%, T 87.5%

  # dropout
  #r = 0    # 85.9%, V 87.1%, T 93.8%
  #r = 5e-4 # 87.5%, V 86.8%, T 93.6%
    
  # dropout + overfitting
  #r = 5e-4 # 100.0%, V 80.6%, T 87.9%

  # learning rate decay
  #r = 0    # 83.6%, V 87.2%, T 93.7%
  r = 5e-4 # 86.7%, V 87.0%, T 93.3%

  # Add the regularization term to the loss.
  loss += r * regularizers

  #  learning rate decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 10000, 0.96, staircase=True)

  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(layer2v)
  test_prediction = tf.nn.softmax(layer2t)

In [41]:
train_dataset2 = train_dataset[:batch_size*10]
train_labels2 = train_labels[:batch_size*10]
print([train_dataset.shape, train_labels.shape])
print([train_dataset2.shape, train_labels2.shape])

[(200000, 784), (200000, 10)]
[(1280, 784), (1280, 10)]


In [67]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.473601
Minibatch accuracy: 13.3%
Validation accuracy: 39.6%
Minibatch loss at step 500: 0.642134
Minibatch accuracy: 85.9%
Validation accuracy: 84.6%
Minibatch loss at step 1000: 0.732804
Minibatch accuracy: 82.8%
Validation accuracy: 85.3%
Minibatch loss at step 1500: 0.493587
Minibatch accuracy: 91.4%
Validation accuracy: 86.4%
Minibatch loss at step 2000: 0.490094
Minibatch accuracy: 91.4%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 0.472273
Minibatch accuracy: 89.8%
Validation accuracy: 86.4%
Minibatch loss at step 3000: 0.546916
Minibatch accuracy: 85.9%
Validation accuracy: 86.5%
Test accuracy: 93.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
